In [1]:
import pandas as pd
import numpy as np

In [2]:
data_files = ['Data/working.txt', 'Data/Customer_review_data/Apex AD2600 Progressive-scan DVD player.txt']

In [8]:
with open(data_files[0], 'r') as file:
    text = file.read()
    reviews = text.strip().split('\n')
    refined_reviews = []
    
    for review in reviews:
        parts = review.split('##')
        
        if len(parts) > 1:
            tags = parts[0].strip().split(',')
            content = parts[1].strip() 
        else:
            tags = []
            content = parts
            
        refined_reviews.append((tags, content))

    for i in refined_reviews:
        print(i)
        print('\n')



([], ['[t] incredibe price / performance . '])


([''], "i 've owned 6 or 7 dvd players since 1998 .")


(['dvd player[+3][p]'], 'this is by far the nicest one , in so many ways .')


(['look[+3]', 'panel button layout[+3]', 'feature[+2]'], "it 's very sleek looking with a very good front panel button layout , and it has a great feature set .")


(['forward[+2]', ' rewind[+2]'], "its fast-forward and rewind work much more smoothly and consistently than those of other models i 've had .")


(['video format[+3]'], 'it plays alternate video formats ( vcds , svcds , cvds ) very well .')


(['price[+3]'], 'and amazon.com has it for such a great price -- how can you go wrong ?')


([''], "i 've bought one for the den and will buy a 2nd one for the bedroom .")


([''], 'strengths are well listed by other reviewers .')


(['play[+2]', ' dvd-r[+1]'], 'what got me to buy was the reviewer that said it would play dvd-rs fill of files ( e. , mp3s ) .')


